Patricia Trie Implementation


In [ ]:
import re
import pandas as pd
import hashlib

In [ ]:
class PatriciaTrieNode:
    def __init__(self):
        self.children = {}
        self.is_end_of_word = False

class PatriciaTrie:
    def __init__(self):
        self.root = PatriciaTrieNode()

    def insert(self, word):
        node = self.root
        while word:
            for key in node.children.keys():
                common_prefix = self._common_prefix(word, key)
                if common_prefix:
                    if common_prefix == key:
                        node = node.children[key]
                        word = word[len(common_prefix):]
                    else:
                        existing_node = node.children.pop(key)
                        new_node = PatriciaTrieNode()
                        new_node.children[key[len(common_prefix):]] = existing_node
                        node.children[common_prefix] = new_node
                        node = new_node
                        word = word[len(common_prefix):]
                        break
            else:
                node.children[word] = PatriciaTrieNode()
                node = node.children[word]
                word = ''
        node.is_end_of_word = True

    def count_frequency(self, sequence, pattern):
        count = 0
        pattern_len = len(pattern)
        for i in range(len(sequence) - pattern_len + 1):
            if sequence[i:i + pattern_len] == pattern:
                count += 1
        return count

    def _common_prefix(self, str1, str2):
        min_len = min(len(str1), len(str2))
        for i in range(min_len):
            if str1[i] != str2[i]:
                return str1[:i]
        return str1[:min_len]

def count_pattern_in_exons_with_patricia_trie(sequence, exon_ranges, pattern):
    trie = PatriciaTrie()
    total_count = 0
    for start, end in exon_ranges:
        segment = sequence[start-1:end]  # Adjusting for 0-based indexing
        total_count += trie.count_frequency(segment, pattern)
    return total_count


 Trie Implementation

In [ ]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_end_of_word = False
        self.frequency = 0

class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.is_end_of_word = True
        node.frequency += 1

    def search(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                return 0
            node = node.children[char]
        return node.frequency if node.is_end_of_word else 0

def count_pattern_in_exons(sequence, exon_ranges, pattern):
    trie = Trie()
    pattern_len = len(pattern)
    for start, end in exon_ranges:
        segment = sequence[start-1:end]  # Adjusting for 0-based indexing
        for i in range(len(segment) - pattern_len + 1):
            substring = segment[i:i + pattern_len]
            trie.insert(substring)
    return trie.search(pattern)


BWT Implementation

In [ ]:
def bwt_transform(sequence):
    sequence = sequence + "$"  # Add end of string marker
    table = sorted(sequence[i:] + sequence[:i] for i in range(len(sequence)))
    return ''.join(row[-1] for row in table)

def bwt_search(bwt, pattern):
    suffix_array = sorted(range(len(bwt)), key=lambda i: bwt[i:])
    first_col = ''.join(sorted(bwt))

    counts = {}
    for char in bwt:
        if char in counts:
            counts[char] += 1
        else:
            counts[char] = 1

    first_occurrence = {}
    total = 0
    for char in sorted(counts.keys()):
        first_occurrence[char] = total
        total += counts[char]

    l, r = 0, len(bwt) - 1
    for char in reversed(pattern):
        if char in first_occurrence:
            l = first_occurrence[char] + bwt[:l].count(char)
            r = first_occurrence[char] + bwt[:r+1].count(char) - 1
        else:
            return 0

    return r - l + 1

def count_pattern_in_exons_with_bwt(sequence, exon_ranges, pattern):
    total_count = 0
    for start, end in exon_ranges:
        segment = sequence[start-1:end]  # Adjusting for 0-based indexing
        bwt = bwt_transform(segment)
        total_count += bwt_search(bwt, pattern)
    return total_count


Bloom Filter Implementation

In [ ]:
import hashlib

class CountingBloomFilter:
    def __init__(self, size, num_hashes):
        self.size = size
        self.num_hashes = num_hashes
        self.bloom = [0] * size

    def _hashes(self, item):
        hashes = []
        for i in range(self.num_hashes):
            hash_result = int(hashlib.md5((item + str(i)).encode()).hexdigest(), 16)
            hashes.append(hash_result % self.size)
        return hashes

    def add(self, item):
        for hash_val in self._hashes(item):
            self.bloom[hash_val] += 1

    def count(self, item):
        return min(self.bloom[hash_val] for hash_val in self._hashes(item))

def find_pattern_frequency(sequence, pattern, bloom_size, num_hashes):
    bloom_filter = CountingBloomFilter(bloom_size, num_hashes)
    kmer_size = len(pattern)
    for i in range(len(sequence) - kmer_size + 1):
        kmer = sequence[i:i+kmer_size]
        bloom_filter.add(kmer)
    return bloom_filter.count(pattern)

def count_pattern_in_exons_with_bloom_filter(sequence, exon_ranges, pattern, bloom_size, num_hashes):
    total_count = 0
    for start, end in exon_ranges:
        segment = sequence[start-1:end]  # Adjusting for 0-based indexing
        total_count += find_pattern_frequency(segment, pattern, bloom_size, num_hashes)
    return total_count


Suffix Array Implementation

In [ ]:
def build_suffix_array(text):
    suffixes = [(text[i:], i) for i in range(len(text))]
    suffixes.sort()
    suffix_array = [suffix[1] for suffix in suffixes]
    return suffix_array

def suffix_array_search(sequence, pattern, suffix_array):
    left = 0
    right = len(suffix_array) - 1
    pattern_length = len(pattern)
    count = 0

    while left <= right:
        mid = (left + right) // 2
        start_index = suffix_array[mid]
        substring = sequence[start_index:start_index + pattern_length]

        if pattern == substring:
            count += 1
            l, r = mid - 1, mid + 1
            while l >= 0 and sequence[suffix_array[l]:suffix_array[l] + pattern_length] == pattern:
                count += 1
                l -= 1
            while r < len(suffix_array) and sequence[suffix_array[r]:suffix_array[r] + pattern_length] == pattern:
                count += 1
                r += 1
            break
        elif pattern < substring:
            right = mid - 1
        else:
            left = mid + 1

    return count

def count_pattern_in_exons_with_suffix_array(sequence, exon_ranges, pattern):
    total_count = 0
    for start, end in exon_ranges:
        segment = sequence[start-1:end]  # Adjusting for 0-based indexing
        suffix_array = build_suffix_array(segment)
        total_count += suffix_array_search(segment, pattern, suffix_array)
    return total_count


Wavelet Tree Implementation

In [ ]:
class WaveletTree:
    def __init__(self, data, alphabet=None):
        if alphabet is None:
            alphabet = sorted(set(data))
        self.alphabet = alphabet
        self.mid = len(alphabet) // 2
        self.left = self.right = None
        self.bit_vector = []

        if len(data) == 0 or len(alphabet) <= 1:  # Base case for recursion
            return

        left_alphabet = alphabet[:self.mid]
        right_alphabet = alphabet[self.mid:]

        left_data = []
        right_data = []

        for char in data:
            if char in left_alphabet:
                self.bit_vector.append(0)
                left_data.append(char)
            else:
                self.bit_vector.append(1)
                right_data.append(char)

        if left_data:
            self.left = WaveletTree(left_data, left_alphabet)
        if right_data:
            self.right = WaveletTree(right_data, right_alphabet)

    def rank(self, char, index):
        if len(self.alphabet) <= 1:
            return index + 1

        if char in self.alphabet[:self.mid]:
            return self.left.rank(char, self.bit_vector[:index + 1].count(0) - 1)
        else:
            return self.right.rank(char, self.bit_vector[:index + 1].count(1) - 1)

    def range_query(self, char, start, end):
        return self.rank(char, end) - self.rank(char, start - 1)

def wavelet_tree_search(sequence, pattern):
    wavelet_tree = WaveletTree(sequence)
    count = 0
    for i in range(len(sequence) - len(pattern) + 1):
        if all(wavelet_tree.range_query(pattern[j], i + j, i + j) > 0 for j in range(len(pattern))):
            count += 1
    return count

def count_pattern_in_exons_with_wavelet_tree(sequence, exon_ranges, pattern):
    total_count = 0
    for start, end in exon_ranges:
        segment = sequence[start-1:end]  # Adjusting for 0-based indexing
        total_count += wavelet_tree_search(segment, pattern)
    return total_count


DFA Implementation

In [ ]:
# DFA Implementation
class DFA:
    def __init__(self, pattern):
        self.pattern = pattern
        self.states = len(pattern) + 1
        self.alphabet = set(pattern)
        self.transition_table = self.build_transition_table()

    def build_transition_table(self):
        transition_table = {}
        for state in range(self.states):
            for char in self.alphabet:
                next_state = min(self.states - 1, state + 1)
                while next_state > 0 and self.pattern[next_state - 1] != char:
                    next_state -= 1
                transition_table[(state, char)] = next_state
        return transition_table

    def search(self, text):
        state = 0
        occurrences = []
        for i, char in enumerate(text):
            if (state, char) in self.transition_table:
                state = self.transition_table[(state, char)]
            else:
                state = 0

            if state == self.states - 1:
                occurrences.append(i - len(self.pattern) + 1)
                state = 0  # Reset state after finding a match
        return occurrences

def count_pattern_in_exons_with_dfa(sequence, exon_ranges, pattern):
    dfa = DFA(pattern)
    total_count = 0
    for start, end in exon_ranges:
        segment = sequence[start-1:end]  # Adjusting for 0-based indexing
        total_count += len(dfa.search(segment))
    return total_count


KMP Implementation

In [ ]:
def kmp_preprocess(pattern):
    lps = [0] * len(pattern)
    length = 0
    i = 1

    while i < len(pattern):
        if pattern[i] == pattern[length]:
            length += 1
            lps[i] = length
            i += 1
        else:
            if length != 0:
                length = lps[length - 1]
            else:
                lps[i] = 0
                i += 1

    return lps

def kmp_search_in_ranges(sequence, pattern, ranges):
    lps = kmp_preprocess(pattern)
    positions = []

    for start, end in ranges:
        i = start  # index for sequence
        j = 0  # index for pattern
        while i < end:
            if pattern[j] == sequence[i]:
                i += 1
                j += 1

            if j == len(pattern):
                positions.append(i - j)
                j = lps[j - 1]
            elif i < end and pattern[j] != sequence[i]:
                if j != 0:
                    j = lps[j - 1]
                else:
                    i += 1

    return positions

def count_pattern_in_exons_with_kmp(sequence, exon_ranges, pattern):
    positions = kmp_search_in_ranges(sequence, pattern, exon_ranges)
    return len(positions)


In [ ]:
# GENSCAN output parsing
def parse_genscan_output(genscan_output):
    pattern = re.compile(r'\s+\d+\.\d+\s+\w+\s+[+-]\s+(\d+)\s+(\d+)\s+\d+\s+\d\s+\d\s+(\d*)\s+(\d*)\s+(\d*)\s+\d*\.\d*\s+\d*\.\d*')
    matches = pattern.findall(genscan_output)
    filtered_matches = [match for match in matches if all(match[2:])]
    columns = ['Begin', 'End', 'Ph', 'I/Ac', 'Do/T']
    data = pd.DataFrame(filtered_matches, columns=columns)
    data[['Begin', 'End']] = data[['Begin', 'End']].apply(pd.to_numeric, errors='coerce')
    exon_ranges = list(zip(data['Begin'], data['End']))
    return exon_ranges


In [ ]:
# GENSCAN output(example output of exon ranges data from genscan server)
genscan_output = """
Gn.Ex Type S .Begin ...End .Len Fr Ph I/Ac Do/T CodRg P.... Tscr..

----- ---- - ------ ------ ---- -- -- ---- ---- ----- ----- ------



 1.01 Sngl +   1127   3889 2763  1  0   60   48  2946 0.998 280.71

 1.02 PlyA +   4392   4397    6                               1.05



 2.03 PlyA -   4549   4544    6                               1.05

 2.02 Term -   5235   5080  156  0  0   44   43   145 0.785   3.53

 2.01 Intr -   7099   6961  139  0  1   86   81    66 0.905   6.17

"""

fasta_file_path =r"C:\Users\yaaju\Downloads\mrnas_by_gene\AR\NM_000044.6.fna"
pattern="CAG"
def load_fasta(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    header = lines[0].strip()
    sequence = ''.join(lines[1:]).replace('\n', '')
    return sequence
sequence = load_fasta(fasta_file_path)
exon_ranges=parse_genscan_output(genscan_output)


print(exon_ranges)

In [ ]:


fasta_file_path =r"C:\Users\yaaju\Downloads\test_atxn1.fasta"
#pattern="CCG"
def load_fasta(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    header = lines[0].strip()
    sequence = ''.join(lines[1:]).replace('\n', '')
    return sequence
sequence = load_fasta(fasta_file_path)
exon_ranges=parse_genscan_output(genscan_output)




print(exon_ranges)

In [ ]:
pattern="TGC" #declaring the pattern of the respective gene
print(exon_ranges)

bwt_count = count_pattern_in_exons_with_bwt(sequence, exon_ranges, pattern)
print(f"Count of '{pattern}' using BWT: {bwt_count}")
trie_count=count_pattern_in_exons(sequence, exon_ranges, pattern)
print(f"Count of '{pattern}' using trie: {trie_count}")
    # Bloom Filter
bloom_size = 10000  # Adjust the size based on your needs
num_hashes = 3      # Number of hash functions
bloom_filter_count = count_pattern_in_exons_with_bloom_filter(sequence, exon_ranges, pattern, bloom_size, num_hashes)
print(f"Count of '{pattern}' using Bloom Filter: {bloom_filter_count}")

    # Suffix Array
suffix_array_count = count_pattern_in_exons_with_suffix_array(sequence, exon_ranges, pattern)
print(f"Count of '{pattern}' using Suffix Array: {suffix_array_count}")

    # Wavelet Tree
wavelet_tree_count = count_pattern_in_exons_with_wavelet_tree(sequence, exon_ranges, pattern)
print(f"Count of '{pattern}' using Wavelet Tree: {wavelet_tree_count}")

    # DFA
dfa_count = count_pattern_in_exons_with_dfa(sequence, exon_ranges, pattern)
print(f"Count of '{pattern}' using DFA: {dfa_count}")

    # KMP
kmp_count = count_pattern_in_exons_with_kmp(sequence, exon_ranges, pattern)
print(f"Count of '{pattern}' using KMP: {kmp_count}")

Patrici_c_trie=count_pattern_in_exons_with_patricia_trie(sequence, exon_ranges, pattern)
print(f"Count of '{pattern}' using patricia: {Patrici_c_trie}")
print(exon_ranges,bwt_count,trie_count,bloom_filter_count,suffix_array_count,wavelet_tree_count,dfa_count,kmp_count,Patrici_c_trie )
print(sequence)